In [1]:
try:
    from medcat.cat import CAT
except:
    print("WARNING: Runtime will restart automatically and please run other cells thereafter.")
    exit()

/home/vmadmin/medcat_venv/lib/python3.8/site-packages/medcat/cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import pandas as pd
import numpy as np
import json
import os
import spacy
import logging

from medcat.cat import CAT
from medcat.cdb import CDB
from medcat.config import Config
from medcat.vocab import Vocab
from medcat.meta_cat import MetaCAT
from medcat.preprocessing.tokenizers import TokenizerWrapperBPE
from tokenizers import ByteLevelBPETokenizer
import dask.dataframe as dd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

In [27]:
#'increase size of notebook'
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# os.chdir('/content/drive/My Drive/Colab Notebooks/mimic-iii-clinical-database-1.4/src')
path = "/home/vmadmin/Anchor_and_Span/src/"
if os.getcwd() != path:
    os.chdir(path)

In [4]:
!ls .

__pycache__	explore.ipynb	  mydask.png	   test_linking.py
anchor_span.py	medcat_ann.ipynb  scisp_ann.ipynb  utils.py
annotate.py	mstext_ann.ipynb  scisp_ann.py	   visualize_annotations.ipynb
creds		mstext_ann.py	  subscription


In [5]:
data_read_dask = dd.read_csv('../../mimic-iii/cleaned/notes/NOTESEVENTS_0.csv', 
                        encoding="utf-8", engine="python", dtype={"CLEANED_TEXT": str})
data_read_dask.head(n=2, compute=True)

index  \
0      0   
1      1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [6]:
nlp = spacy.load('en_core_web_sm')

In [7]:
#using spacy multiprocess pip function to multiprocess data
def spacyProcess(data, dask=False):
    if dask:
        data.loc[:,"PROCESSED_TEXT"] = data['CLEANED_TEXT'].apply(lambda x:nlp(x))
        return data
    documents = nlp.pipe(data['CLEANED_TEXT'].tolist())
    return documents

In [9]:
start, end = 10000, 12500

In [10]:
data_read_ = data_read_dask.compute()

In [11]:
if len(data_read_) > end:
    end = len(data_read_)
data_read = data_read_[start:end]

In [12]:
spacy_processed_data = spacyProcess(data=data_read)

In [13]:
document_sent_map = {}
sentences = []
sent_id = 0
for doc_id,doc in enumerate(spacy_processed_data):
    doc_sents = doc.sents
    doc_sents_ids = []
    for sent in doc_sents:
        sentences.append(" ".join([m.text for m in sent]))
        doc_sents_ids.append(sent_id)
        sent_id += 1
    document_sent_map[doc_id] = (doc_sents_ids[0], doc_sents_ids[-1]+1)
data_read_sp = pd.DataFrame({'CLEANED_TEXT':sentences})

In [14]:
dict([[k,v] for k,v in document_sent_map.items()][:5])

{0: (0, 227), 1: (227, 307), 2: (307, 431), 3: (431, 554), 4: (554, 679)}

In [15]:
data_read_sp.head()

,CLEANED_TEXT
0,Admission Date : 2116 - 11 - 16 Discharge Date : 2116 - 11 - 20 Date of Birth : 2035 - 12 - 10 Sex : M Service : MEDICINE Allergies : Sulfonamides / Iodine ;
1,Iodine Containing / Influenza Virus Vaccine Attending : First Name3 ( LF ) 6195
2,"Chief Complaint : Found down , hypotensive in the ED Major Surgical or Invasive Procedure : None History of Present Illness : 80 yo man with history of neuroendocrine tumor s / p colostomy and chemotherapy , paroximal atrial fibrillation on coumadin who presents being found down by wife ."
3,Name ( NI ) had been feeling weak for 4 days .
4,"Then today , he was taking his sock off when he slipped off the bed and fell to floor ."


In [16]:
# format the df to match: required input data for multiprocessing = [(doc_id, doc_text), (doc_id, doc_text), ...]
def data_iterator_df(data):
    for id, row in data[['CLEANED_TEXT']].iterrows():
        yield (id, str(row['CLEANED_TEXT']))

In [17]:
cat = CAT.load_model_pack("../models/umls_sm_pt2ch_533bab5115c6c2d6.zip")

2023-07-15 14:46:35,430 - Found an existing unziped model pack at: ../models/umls_sm_pt2ch_533bab5115c6c2d6, the provided zip will not be touched.
2023-07-15 14:46:35,432 - Loading model pack with dill format
2023-07-15 14:46:35,432 - Reading CDB data from ../models/umls_sm_pt2ch_533bab5115c6c2d6/cdb.dat
2023-07-15 14:46:50,356 - You have MedCAT version '1.8.0' installed while the CDB was exported by MedCAT version '1.3.1.dev315',
which may or may not work. If you experience any compatibility issues, please reinstall MedCAT
or download the compatible model.
/home/vmadmin/medcat_venv/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.

In [18]:
# Set a batch size to control for the variablity between document sizes
batch_size_chars = 500000 # Batch size (BS) in number of characters

# Run model
results = cat.multiprocessing(data_iterator_df(data_read_sp),  # Formatted data
                              # batch_size_chars = batch_size_chars,
                              nproc=8) # Number of processors

2023-07-15 14:47:06,998 - Annotated until now: 0 docs; Current BS: 65342 docs; Elapsed time: 0.04 minutes
2023-07-15 14:49:24,370 - Annotated until now: 65342 docs; Current BS: 65238 docs; Elapsed time: 2.32 minutes
2023-07-15 14:51:40,169 - Annotated until now: 130580 docs; Current BS: 65790 docs; Elapsed time: 4.59 minutes
2023-07-15 14:54:01,984 - Annotated until now: 196370 docs; Current BS: 68595 docs; Elapsed time: 6.95 minutes
2023-07-15 14:56:17,798 - Annotated until now: 264965 docs; Current BS: 35798 docs; Elapsed time: 9.22 minutes


In [19]:
len(results)

300763

In [20]:
#find the token id/position of an entity given its char offset from start of a sentence and the sentence
def fetch_entitis_span_pos(entity, sentence, extractor):
    sentence_tokenized = dict([(i, j) for i, j in enumerate(sentence.split())])
    token_ids_cumulative_length = {}
    if extractor == 'MSTA4H':
        char_offset, entity_len, entity_text = entity.offset, entity.length, entity.text
    if extractor == 'MEDCAT':
        char_offset, entity_len, entity_text = entity[0], entity[1]-entity[0], entity[2]
    curr_offset = 0
    span, end_span_found = [], False
    if char_offset == 0:
        span.append(0)

    for i, j in sentence_tokenized.items():
        if curr_offset >= char_offset + entity_len:
            span.append(i)
            end_span_found = True
        else:
            curr_offset = curr_offset + len(j) + 1
            if curr_offset == char_offset:
                span.append(i + 1)
        if end_span_found == True:
            if len(span) != 2:
                print("Failed to detect span position {} {}".format(entity_text, span))
            try:
                assert len(span) == 2
                break
            except Exception as e:
                return entity_text, span
    return entity_text, span

In [21]:
results_copy = dict(sorted(results.items(), key=lambda x:x[0]))

In [22]:
dataset_ann = []
for doc_id, document_sent_range in document_sent_map.items():
    doc_ann = {}
    doc_entities = []
    sents = []
    for sent_id, doc in enumerate(results_copy):
        if sent_id in range(document_sent_range[0], document_sent_range[-1]):
            sents.append(sentences[sent_id].split())
            for k,v in results_copy[doc]['entities'].items():
                entity = {}
                _entity_ = (v['start'], v['end'], v['source_value'])
                entity_text, entity_span_pos = fetch_entitis_span_pos(_entity_, sentences[sent_id], "MEDCAT")
                try:
                    assert len(entity_span_pos) == 2
                except AssertionError:
                    entity['pos'] = entity_span_pos[1]
                entity['name'] = entity_text
                entity['sent_id'] = sent_id
                entity['char_position'] = [v['start'], v['end']]
                entity['token_pos'] = entity_span_pos[1]
                entity['score'] = v['acc']
                entity['linked_entities'] = []
                linked_entity = {}
                for m,n in zip(v['types'], v['type_ids']):
                    linked_entity['cui'] = v['cui']
                    linked_entity['type'] = m
                    linked_entity['type_id'] = n
                    entity['linked_entities'].append(linked_entity)
                if entity:
                    doc_entities.append(entity)
        if sent_id == (document_sent_range[-1]-1):
            break
    doc_ann['Entities'] = doc_entities
    doc_ann['Sents'] = sents
    dataset_ann.append(doc_ann)

Failed to detect span position LDH [1]
Failed to detect span position 29PM [34]
Failed to detect span position 20PM [42]
Failed to detect span position 30AM [28]
Failed to detect span position 30AM [30]
Failed to detect span position 30am [12]
Failed to detect span position 18AM [12]
Failed to detect span position 29PM [11]
Failed to detect span position 29PM [22]
Failed to detect span position Patient lives alone [13]
Failed to detect span position Tobacco [1]
Failed to detect span position ETOH [5]
Failed to detect span position 20PM [45]
Failed to detect span position 20PM [14]
Failed to detect span position 20PM [42]
Failed to detect span position 20PM [62]
Failed to detect span position Tobacco [1]
Failed to detect span position ETOH [5]
Failed to detect span position 31AM [10]
Failed to detect span position 19AM [20]
Failed to detect span position 20PM [9]
Failed to detect span position 19AM [18]
Failed to detect span position 19AM [5]
Failed to detect span position 19AM [10]
Fai

Failed to detect span position HTN [6]
Failed to detect span position HTN [20]
Failed to detect span position 32PM [20]
Failed to detect span position 23PM [47]
Failed to detect span position 32PM [65]
Failed to detect span position 23PM [95]
Failed to detect span position 32PM [114]
Failed to detect span position 23PM [141]
Failed to detect span position LDH [158]
Failed to detect span position 25PM [48]
Failed to detect span position 25PM [68]
Failed to detect span position 25PM [1]
Failed to detect span position 25PM [57]
Failed to detect span position 30pm [16]
Failed to detect span position Diabetic ketoacidosis [8]
Failed to detect span position Dry gangrene [10]
Failed to detect span position Return to ED [15]
Failed to detect span position Follow up [2]
Failed to detect span position 30am [23]
Failed to detect span position Constitutional [1]
Failed to detect span position Blurry Vision [6]
Failed to detect span position ENT [18]
Failed to detect span position Respiratory [4]
F

Failed to detect span position care [14]
Failed to detect span position LAD [43]
Failed to detect span position 30AM [6]
Failed to detect span position 30AM [9]
Failed to detect span position 20PM [9]
Failed to detect span position LDH [49]
Failed to detect span position LDH [25]
Failed to detect span position 20PM [33]
Failed to detect span position 10PM [79]
Failed to detect span position 15PM [10]
Failed to detect span position LDH [4]
Failed to detect span position 22PM [35]
Failed to detect span position 13PM [17]
Failed to detect span position Degenerative disc disease [14]
Failed to detect span position 30AM [6]
Failed to detect span position 30AM [9]
Failed to detect span position 30AM [8]
Failed to detect span position 30AM [28]
Failed to detect span position abd [3]
Failed to detect span position Rhythm [5]
Failed to detect span position CAD [1]
Failed to detect span position Type 1 [10]
Failed to detect span position CAD [13]
Failed to detect span position Ventricular tachyc

Failed to detect span position hip [21]
Failed to detect span position 30PM [16]
Failed to detect span position 30PM [20]
Failed to detect span position 30PM [20]
Failed to detect span position 24PM [11]
Failed to detect span position 24PM [8]
Failed to detect span position Mental Status [4]
Failed to detect span position Cranial Nerves [2]
Failed to detect span position LDH [4]
Failed to detect span position hemolyzed [4]
Failed to detect span position 30PM [20]
Failed to detect span position 32PM [16]
Failed to detect span position Zoloft [5]
Failed to detect span position Atenolol 25 mg [11]
Failed to detect span position Lisinopril 40 mg [15]
Failed to detect span position Crestor [17]
Failed to detect span position Lantus [21]
Failed to detect span position Metformin 1000 mg [26]
Failed to detect span position ASA [28]
Failed to detect span position murmur [10]
Failed to detect span position edema [34]
Failed to detect span position varicosities [36]
Failed to detect span position

Failed to detect span position 24PM [32]
Failed to detect span position 24PM [10]
Failed to detect span position 24PM [19]
Failed to detect span position 24PM [43]
Failed to detect span position 24PM [8]
Failed to detect span position 12PM [19]
Failed to detect span position 18PM [38]
Failed to detect span position LDH [22]
Failed to detect span position 6 - mm [6]
Failed to detect span position 10PM [10]
Failed to detect span position LDH [2]
Failed to detect span position difficulty breathing [19]
Failed to detect span position fever [20]
Failed to detect span position 25PM [74]
Failed to detect span position digoxin [1]
Failed to detect span position Monitor [1]
Failed to detect span position 30PM [16]
Failed to detect span position 30PM [20]
Failed to detect span position 30PM [9]
Failed to detect span position 30PM [10]
Failed to detect span position 30PM [18]
Failed to detect span position 18AM [28]
Failed to detect span position 18AM [8]
Failed to detect span position 18AM [26]


...

Failed to detect span position 30AM [55]
Failed to detect span position 20pm [13]
Failed to detect span position Diabetes [5]
Failed to detect span position Hypertension [6]
Failed to detect span position Urinary tract infections [9]
Failed to detect span position PVD [1]
Failed to detect span position Dementia [2]
Failed to detect span position PUD [3]
Failed to detect span position Iron def anemia [8]
Failed to detect span position 30PM [59]
Failed to detect span position 30PM [43]
Failed to detect span position 30PM [52]
Failed to detect span position 30PM [60]
Failed to detect span position 30PM [7]
Failed to detect span position 30PM [10]
Failed to detect span position 30PM [11]
Failed to detect span position 18PM [30]
Failed to detect span position 30PM [31]
Failed to detect span position 30PM [40]
Failed to detect span position 30am [7]
Failed to detect span position Simethicone [27]
Failed to detect span position 30AM [9]
Failed to detect span position 30AM [10]
Failed to detec

Failed to detect span position 30PM [12]
Failed to detect span position 30PM [13]
Failed to detect span position 30PM [26]
Failed to detect span position 30PM [18]
Failed to detect span position Fever [1]
Failed to detect span position Increased shortness of breath [9]
Failed to detect span position Incision [2]
Failed to detect span position Labs [1]
Failed to detect span position 30PM [67]
Failed to detect span position 30PM [9]
Failed to detect span position 15PM [9]
Failed to detect span position 15PM [10]
Failed to detect span position 30PM [8]
Failed to detect span position 15PM [17]
Failed to detect span position 15PM [35]
Failed to detect span position 15PM [44]
Failed to detect span position 30PM [68]
Failed to detect span position 15PM [20]
Failed to detect span position 15PM [9]
Failed to detect span position 15PM [26]
Failed to detect span position 30AM [12]
Failed to detect span position Tobacco [1]
Failed to detect span position ETOH [5]
Failed to detect span position 30A

Failed to detect span position 18AM [51]
Failed to detect span position 18AM [6]
Failed to detect span position 18AM [30]
Failed to detect span position presents [61]
Failed to detect span position 30PM [124]
Failed to detect span position 17AM [8]
Failed to detect span position 30PM [8]
Failed to detect span position 30PM [26]
Failed to detect span position 30PM [60]
Failed to detect span position 30pm [11]
Failed to detect span position LAD [12]
Failed to detect span position Carotid Bruits [17]
Failed to detect span position Varicosities [48]
Failed to detect span position 27PM [6]
Failed to detect span position 27PM [26]
Failed to detect span position 27PM [9]
Failed to detect span position 19PM [10]
Failed to detect span position 19PM [30]
Failed to detect span position 19PM [38]
Failed to detect span position 19PM [52]
Failed to detect span position 19PM [63]
Failed to detect span position 19PM [12]
Failed to detect span position 19PM [9]
Failed to detect span position 19PM [9]
F

Failed to detect span position 30PM [26]
Failed to detect span position 30PM [6]
Failed to detect span position 30PM [20]
Failed to detect span position 30PM [8]
Failed to detect span position 30PM [15]
Failed to detect span position 30PM [22]
Failed to detect span position 30PM [10]
Failed to detect span position LDH [18]
Failed to detect span position 30AM [17]
Failed to detect span position Flagyl [1]
Failed to detect span position Aspirin [21]
Failed to detect span position 5 mm [3]
Failed to detect span position 30AM [19]
Failed to detect span position 24PM [11]
Failed to detect span position 30AM [8]
Failed to detect span position 30AM [23]
Failed to detect span position 30AM [37]
Failed to detect span position 24PM [8]
Failed to detect span position 24PM [23]
Failed to detect span position 30AM [8]
Failed to detect span position 24PM [32]
Failed to detect span position 24PM [40]
Failed to detect span position 30AM [11]
Failed to detect span position 24PM [1]
Failed to detect spa

Failed to detect span position LDH [7]
Failed to detect span position LDH [48]
Failed to detect span position TSH [7]
Failed to detect span position possibly [9]
Failed to detect span position EKG [23]
Failed to detect span position forgets [47]
Failed to detect span position unable [77]
Failed to detect span position knee jerk [99]
Failed to detect span position LDH [22]
Failed to detect span position 30PM [22]
Failed to detect span position 30AM [36]
Failed to detect span position 30AM [58]
Failed to detect span position 30AM [90]
Failed to detect span position 30AM [100]
Failed to detect span position 27PM [35]
Failed to detect span position 27PM [28]
Failed to detect span position cell [21]
Failed to detect span position 29PM [69]
Failed to detect span position 21PM [76]
Failed to detect span position LDH [15]
Failed to detect span position 15PM [17]
Failed to detect span position 15PM [25]
Failed to detect span position 15PM [33]
Failed to detect span position 28PM [44]
Failed to 

In [23]:
dataset_ann[0]

{'Entities': [{'name': 'Date of Birth',
   'sent_id': 0,
   'char_position': [64, 77],
   'token_pos': [16, 19],
   'score': 0.3482390952942088,
   'linked_entities': [{'cui': 'C0421451',
     'type': 'Finding',
     'type_id': 'T033'}]},
  {'name': 'Sex',
   'sent_id': 0,
   'char_position': [95, 98],
   'token_pos': [25, 26],
   'score': 0.99,
   'linked_entities': [{'cui': 'C1522384',
     'type': 'Organism Attribute',
     'type_id': 'T032'}]},
  {'name': 'MEDICINE',
   'sent_id': 0,
   'char_position': [113, 121],
   'token_pos': [30, 31],
   'score': 0.8269954570620657,
   'linked_entities': [{'cui': 'C0025118',
     'type': 'Biomedical Occupation or Discipline',
     'type_id': 'T091'}]},
  {'name': 'Sulfonamides',
   'sent_id': 0,
   'char_position': [134, 146],
   'token_pos': [33, 34],
   'score': 0.99,
   'linked_entities': [{'cui': 'C0038760',
     'type': 'Organic Chemical',
     'type_id': 'T109'}]},
  {'name': 'Iodine',
   'sent_id': 0,
   'char_position': [149, 155],
  

In [24]:
def createDir(path):
    dest = path
    if not os.path.exists(path):
        dest = os.makedirs(path)
    return dest

In [25]:
dest_dir = '../anns/medcat/'
createDir(dest_dir)

'../anns/medcat/'

In [26]:
import pickle

annotation_file = 'anns_mult_{}_{}.pkl'.format(start, end)
with open(os.path.join(dest_dir, annotation_file), 'wb') as a:
    pickle.dump(dataset_ann, a, protocol=pickle.HIGHEST_PROTOCOL)
    a.close()